In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2025-02-28T17:59:23.514201-05:00

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 8.10.0

Compiler    : Clang 16.0.0 (clang-1600.0.26.6)
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import sklearn
import gc
import os

In [4]:
%watermark --iversions

pandas : 2.2.3
sklearn: 1.5.2
numpy  : 1.26.4



In [5]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

In [6]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
columns = train.loc[:, train.nunique() > 1].columns
train = train[columns]
test = test[columns]

In [9]:
X = train[train.columns[1:]].values
X_test = test[test.columns[1:]].values
Y = train.label.values
X.shape

(60000, 717)

In [10]:
train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

(60000, 10)

In [11]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=400)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold accuracy: 0.9804166666666667
Fitting fold 2
Fold accuracy: 0.97875
Fitting fold 3
Fold accuracy: 0.97825
Fitting fold 4
Fold accuracy: 0.97775
Fitting fold 5
Fold accuracy: 0.97525
CPU times: user 1h 8min 24s, sys: 2h 40min 23s, total: 3h 48min 48s
Wall time: 12min 48s


In [12]:
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

0.9780833333333333


In [13]:
0.9775833333333334

0.9775833333333334

In [14]:
preds = np.argmax(test_preds, axis=1)
preds

array([7, 2, 1, ..., 4, 5, 6])

In [15]:
test.label.values

array([7, 2, 1, ..., 4, 5, 6])

In [16]:
accuracy_score(test.label.values, preds)

0.9805

In [17]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=400, validation_fraction=None, 
                                           random_state=137, max_depth=15)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

Y_test = test.label.values

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.98375
Fitting fold 2
Fold accuracy: 0.9815
Fitting fold 3
Fold accuracy: 0.9815
Fitting fold 4
Fold accuracy: 0.9818333333333333
Fitting fold 5
Fold accuracy: 0.9780833333333333
0.9813333333333333
0.9824
CPU times: user 2h 25min 16s, sys: 5h 13min 9s, total: 7h 38min 26s
Wall time: 25min 33s
